In [4]:
import socket

s = socket.socket( socket.AF_INET, socket.SOCK_DGRAM )
s.bind( ("134.208.243.41", 5000) ) #輸入你的IP Address

import random
def Dealer():  #洗牌發牌
    pk=[]
    for i in range(1,5):      #  01~04
        for j in range(1,14):  #  01~13
            pk.append( [i,j] )

    pkn=[]
    for i in range(52):
        p=random.randint(0,len(pk)-1) 
        pkn.append( pk[p] )
        del pk[p]

    z=""
    pk=[]
    for j in range(4):
        for i in range(13):
            [a,b]=pkn[j*13+i]
            z=z+"%.2d%.2d" % (a,b)
        pk.append(z)
        z=""

    return(pk)    

In [ ]:
AllPK = Dealer() #洗牌

Vc = Sc = 0
Loss = 1000
P = 1
playerIP = []
PlayedCard = []
Round = 0
while True:
    data, addr = s.recvfrom(1024)
    re = data.decode()
    if (re=='Connecting'):
        r = str('P' + str(P) + ' ' + addr[0]) +':'+ str(addr[1])   #Add Player to PlayerList
        playerIP.append([ addr[0], addr[1], P ])
        P += 1
        print(r + ' is connected')
        print('PlayerList: ')
        print(playerIP)
        
        se = r 
        s.sendto( se.encode(),(addr[0], addr[1]) ) #Send player Info
     
    elif (re[0:12]=='I need Card!'):
        Pn = int(re[12])
        se = AllPK[Pn-1]
        s.sendto( se.encode(),(addr[0], addr[1]) ) #Send Card
    
    elif (re=='Game start?'):
        if (len(playerIP)<4):
            se = 'Not yet'
            s.sendto( se.encode(),(addr[0], addr[1]) )
        else:
            se = 'Game start!'
            s.sendto( se.encode(),(addr[0], addr[1]) )
    
    elif (re=='What round is this?'):
        se = str(Round)
        s.sendto( se.encode(),(addr[0], addr[1]) )
    
    elif (re[0:17]=='PlayCard request!'):

        VS = re[17:]
        V,S = VS.split(',')
        V = int(V)
        S = int(S)
        
        if (V==7):
            PlayedCard.append([V,S])
            Vc,Sc = V,S
            se = 'Card Played'
            s.sendto( se.encode(),(addr[0], addr[1]) )
            
        elif ([V-1,S] in PlayedCard) or ([V+1,S] in PlayedCard):
            PlayedCard.append([V,S])
            Vc,Sc = V,S
            se = 'Card Played'
            s.sendto( se.encode(),(addr[0], addr[1]) )
        else:
            se = 'Cant Play!'
            s.sendto( se.encode(),(addr[0], addr[1]) )
    
    elif (re=='Who is the latest card?'):
        se = "%.2d%.2d" % (Sc,Vc)
        s.sendto( se.encode(),(addr[0], addr[1]) )
            
    elif (re=='Next Round!'):
        Round += 1
        
    elif (re[0:10]=='Game Over!'):
        if(Loss>int(re[11:])):
            Winner = re[10]
            Loss = int(re[11:])
            
    elif (re=='Who is winner?'):     
        se = Winner
        s.sendto( se.encode(),(addr[0], addr[1]) )
        se = str(Loss)
        s.sendto( se.encode(),(addr[0], addr[1]) )
        
    else:
        se = 'Connected'
        s.sendto( se.encode(),(addr[0], addr[1]) ) #Default
        

